In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import urllib2
import json
import time


class WowClassesResources:
    WOW_FORUM_ROOT_URL_EU = 'http://eu.battle.net'
    WOW_FORUM_ROOT_URL_US = 'http://us.battle.net'
    WOW_FORUM_MID_URL = '/forums/en/wow/'
    URL_PAGE_ATTRIBUTE = "?page="

    WOW_CLASS_LIST_EU = [('Death Knight', "874789/"),
                      ('Demon Hunter', "19369494/"),
                      ('Druid', "874790/"),
                      ('Hunter', "874791/"),
                      ('Mage', "874792/"),
                      ('Monk', "6038099/"),
                      ('Paladin', "874793/"),
                      ('Priest', "874794/"),
                      ('Rogue', "874795/"),
                      ('Shaman', "874796/"),
                      ('Warlock', "874929/"),
                      ('Warrior', "874930/")]
    WOW_CLASS_LIST_US = [('Death Knight', "1012662/"),
                      ('Demon Hunter', "22813967/"),
                      ('Druid', "1012663/"),
                      ('Hunter', "1012664/"),
                      ('Mage', "1012760/"),
                      ('Monk', "7379141/"),
                      ('Paladin', "1012668/"),
                      ('Priest', "1012666/"),
                      ('Rogue', "1012667/"),
                      ('Shaman', "1012669/"),
                      ('Warlock', "1012670/"),
                      ('Warrior', "1012759/")]

    WOW_TOPIC_URLS_EU = [(c, WOW_FORUM_ROOT_URL_EU + WOW_FORUM_MID_URL + u) for (c, u) in WOW_CLASS_LIST_EU]

print WowClassesResources.WOW_TOPIC_URLS_EU[0]
url = WowClassesResources.WOW_TOPIC_URLS_EU[0][1]
req = urllib2.Request(url)
response = urllib2.urlopen(req)
html = response.read().decode('utf8')
print html[:100]

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-1ec78f1c27d9>, line 41)

In [3]:
def get_page_content(url):
    attempt_number = 0
    while True:
        # print url
        req = urllib2.Request(url)
        attempt_number += 1
        if attempt_number > 5:
            return None
        try:
            response = urllib2.urlopen(req)
        except urllib2.HTTPError, e:
            print 'HTTPError = ' + str(e.code)
            continue
        except Exception, e:
            print e
            continue
        except error, e:
            print e
            continue
        html = response.read().decode('utf8')
        break
    return html


def topic_scrape(forum_url):
    all_links = []
    i = 1
    while True:
        html = get_page_content(forum_url + WowClassesResources.URL_PAGE_ATTRIBUTE + str(i))
        if html is None:
            return []
        bs_html = BeautifulSoup(html, "html.parser")

        all_link_elements = bs_html.find_all("a", class_="ForumTopic")
        # pprint(all_link_elements)
        link_count = len(all_link_elements)
        print str(i) + ' ' + str(link_count)

        if link_count == 0:
            print 'Exiting'
            break
        for a in all_link_elements:
            # print a
            title = unicode(a.find(class_='ForumTopic-title').get_text()).strip()
            # print title
            all_links.append((a['href'], title))
        i += 1
        time.sleep(10)
    return all_links


def extract_topic(topic_title_url_tuple):
    topic_url_ending = topic_title_url_tuple[0]
    topic_title = topic_title_url_tuple[1]
    pprint('[' + topic_title + '] (' + topic_url_ending + ')')
    all_posts = []
    i = 1
    while True:
        html = get_page_content(
            WowClassesResources.WOW_FORUM_ROOT_URL_EU + topic_url_ending + WowClassesResources.URL_PAGE_ATTRIBUTE + str(
                i))
        if html is None:
            return []
        bs_html = BeautifulSoup(html, "html.parser")

        all_topic_post_elements = bs_html.find_all("div", class_="TopicPost-content")
        # pprint(all_topic_post_elements)
        link_count = len(all_topic_post_elements)
        # print str(i) + ' ' + str(link_count)

        if link_count == 0:
            print 'Exiting'
            break
        for element in all_topic_post_elements:
            # print a
            post_body = unicode(element.find(class_='TopicPost-bodyContent').get_text()).strip()
            post_date = element.find("a", class_='TopicPost-timestamp')['data-tooltip-content']
            author_element = element.find("span", class_='Author-name')
            if author_element is not None:
                # post_author = '<UNKNOWN>'
                if author_element.a is not None:
                    post_author = unicode(author_element.a.get_text()).strip()
                else:
                    post_author = unicode(author_element.get_text()).strip()
                all_posts.append((topic_title, post_date, post_author, post_body))
        i += 1
        time.sleep(3)
    pprint('posts: ' + str(len(all_posts)))
    return all_posts


def save_to_json_file(path, file_name, topic_content_tuple_list, mode):
    with open(path + file_name + '.json', mode) as out_file:
        json.dump(topic_content_tuple_list, out_file)


def read_from_json_file(path, file_name):
    with open(path + file_name + '.json', 'r') as in_file:
        return json.load(in_file)

In [4]:
# READ CURRENT CLASS TO WORK WITH
currentClass = WowClassesResources.WOW_TOPIC_URLS_EU[0]
print currentClass[0]
print currentClass[1]

Death Knight
http://eu.battle.net/forums/en/wow/874789/


In [ ]:
# RUN TOPIC LINK EXTRACTION (ON CURRENT CLASS)
corpora_links = {}
currentClass_links = topic_scrape(currentClass[1])
corpora_links[currentClass[0]] = currentClass_links
pprint(corpora_links[currentClass[0]][:10])

Death Knight
http://eu.battle.net/forums/en/wow/874789/


In [7]:
# SAVE EXTRACTED LINKS
save_to_json_file('Resources/Corpora/EU/Links/', 'corpora_topic_links_' + currentClass[0],
                  corpora_links, 'wb')

In [25]:
# READ LINKS FOR A CLASS FROM FILE
read_corpora_links_instance \
    = read_from_json_file('Resources/Corpora/EU/Links/', 'corpora_topic_links_' + currentClass[0])
pprint(read_corpora_links_instance.keys())
pprint(read_corpora_links_instance[currentClass[0]][:2])

[u'Death Knight']
[[u'/forums/en/wow/topic/10746975071',
  u'[Guide]Death Knight 101 (Updated for 7.03)'],
 [u'/forums/en/wow/topic/10119512036', u'Making a Guide?']]


In [45]:
extracted_topics = []
progress_cursor = 1
for topicEntry in read_corpora_links_instance[currentClass[0]]:
    content = extract_topic(topicEntry)
    extracted_topics.extend(content)

    progress_percent = int(progress_cursor * 100 / len(read_corpora_links_instance[currentClass[0]]))
    print 'PROGRESS: ' + str(progress_percent) + '% (' + str(progress_cursor) + '/' \
          + str(len(read_corpora_links_instance[currentClass[0]])) + ')'

    if progress_cursor % 50 == 0:
        save_to_json_file('Resources/Corpora/EU/Posts/', 'corpora_posts_list_' + currentClass[0],
                          extracted_topics, 'wb')
    progress_cursor += 1

print 'DONE! Total posts found: ' + str(len(extracted_topics))
posts_by_class = {currentClass[0]: extracted_topics}

u'[[Guide]Death Knight 101 (Updated for 7.03)] (/forums/en/wow/topic/10746975071)'


Exiting
'posts: 34'
PROGRESS: 0% (1/11783)
u'[Making a Guide?] (/forums/en/wow/topic/10119512036)'


Exiting
'posts: 1'
PROGRESS: 0% (2/11783)
u'[Frost and the reverse synergy] (/forums/en/wow/topic/17614484421)'


Exiting
'posts: 8'
PROGRESS: 0% (3/11783)
u'[Bring back 2H-Frost DK... in an cosmetic way] (/forums/en/wow/topic/17614111984)'


[Errno 10054] An existing connection was forcibly closed by the remote host


...

Exiting
'posts: 2'
PROGRESS: 29% (3457/11783)
u'[New death knight here] (/forums/en/wow/topic/10494791624)'


Exiting
'posts: 3'
PROGRESS: 29% (3458/11783)
u'[the same old what spec for leveling post] (/forums/en/wow/topic/10494680195)'


Exiting
'posts: 8'
PROGRESS: 29% (3459/11783)
u"[PVP macro's] (/forums/en/wow/topic/10495110729)"


Exiting
'posts: 4'
PROGRESS: 29% (3460/11783)
u'[Rate the Death Knight set above you part 7] (/forums/en/wow/topic/8882098190)'


[Errno 10054] An existing connection was forcibly closed by the remote host


In [ ]:
# SAVE LIST OF POSTS
posts_by_class = {currentClass[0]: extracted_topics}
save_to_json_file('Resources/Corpora/EU/Posts/', 'corpora_posts_' + currentClass[0], posts_by_class, 'wb')

In [10]:
# READ DICTIONARY OF POSTS
posts_by_class = read_from_json_file('Resources/Corpora/EU/Posts/', 'corpora_posts_' + currentClass[0])
print len(posts_by_class[currentClass[0]])

34186
